![MLU Logo](data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Tabular Data - Lecture 1</a>


## Exploratory data analysis

In this notebook, we go through basic steps of exploratory data analysis (EDA), performing initial data investigations to discover patterns, spot anomalies, and look for insights to inform later ML modeling choices.

1. <a href="#1">Read the dataset</a>
2. <a href="#2">Overall Statistics</a>
3. <a href="#3">Univariate Statistics: Basic Plots</a>
4. <a href="#4">Multivariate Statistics: Scatter Plots and Correlations</a>
5. <a href="#5">Handling Missing Values</a>
    * <a href="#51">Drop columns with missing values</a>
    * <a href="#52">Drop rows with missing values</a>
    * <a href="#53">Impute (fill-in) missing values with .fillna()</a>
    * <a href="#54">Impute (fill-in) missing values with sklearn's SimpleImputer</a>
    
__Austin Animal Center Dataset__:

In this exercise, we are working with pet adoption data from __Austin Animal Center__. We have two datasets that cover intake and outcome of animals. Intake data is available from [here](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm) and outcome is from [here](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238). 

In order to work with a single table, we joined the intake and outcome tables using the "Animal ID" column and created a single __review.csv__ file. We also didn't consider animals with multiple entries to the facility to keep our dataset simple. If you want to see the original datasets and the merged data with multiple entries, they are available under data/review folder: Austin_Animal_Center_Intakes.csv, Austin_Animal_Center_Outcomes.csv and Austin_Animal_Center_Intakes_Outcomes.csv.

__Dataset schema:__ 
- __Pet ID__ - Unique ID of pet
- __Outcome Type__ - State of pet at the time of recording the outcome (0 = not placed, 1 = placed). This is the field to predict.
- __Sex upon Outcome__ - Sex of pet at outcome
- __Name__ - Name of pet 
- __Found Location__ - Found location of pet before entered the center
- __Intake Type__ - Circumstances bringing the pet to the center
- __Intake Condition__ - Health condition of pet when entered the center
- __Pet Type__ - Type of pet
- __Sex upon Intake__ - Sex of pet when entered the center
- __Breed__ - Breed of pet 
- __Color__ - Color of pet 
- __Age upon Intake Days__ - Age of pet when entered the center (days)
- __Age upon Outcome Days__ - Age of pet at outcome (days)


## 1. <a name="1">Read the dataset</a>
(<a href="#0">Go to top</a>)

Let's read the dataset into a dataframe, using Pandas.

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
  
df = pd.read_csv('data/review/review_dataset.csv')

print('The shape of the dataset is:', df.shape)

## 2. <a name="2">Overall Statistics</a>
(<a href="#0">Go to top</a>)

We will look at number of rows, columns and some simple statistics of the dataset.

In [ ]:
# Print the first five rows
# NaN means missing data
df.head()

In [ ]:
# Let's see the data types and non-null values for each column
df.info()

In [ ]:
# This prints basic statistics for numerical columns
df.describe()

Let's separate model features and model target.

In [ ]:
print(df.columns)

In [ ]:
model_features = df.columns.drop('Outcome Type')
model_target = 'Outcome Type'

print('Model features: ', model_features)
print('Model target: ', model_target)

We can explore the features set further, figuring out first what features are numerical or categorical. Beware that some integer-valued features could actually be categorical features, and some categorical features could be text features. 

In [ ]:
import numpy as np
numerical_features_all = df[model_features].select_dtypes(include=np.number).columns
print('Numerical columns:',numerical_features_all)

print('')

categorical_features_all = df[model_features].select_dtypes(include='object').columns
print('Categorical columns:',categorical_features_all)


## 3. <a name="3">Basic Plots</a>
(<a href="#0">Go to top</a>)

In this section, we examine our data with plots. Important note: These plots ignore null (missing) values. We will learn how to deal with missing values in the next section.


__Bar plots__: These plots show counts of categorical data fields. __value_counts()__ function yields the counts of each unique value. It is useful for categorical variables.

First, let's look at the distribution of the model target.

In [ ]:
df[model_target].value_counts()

__plot.bar()__ addition to the __value_counts()__ function makes a bar plot of the values.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df[model_target].value_counts().plot.bar()
plt.show()

Now onto the categorical features, exploring number of unique values per feature.

In [ ]:
for c in categorical_features_all: 
    print(df[c].value_counts())
    

Based on the number of unique values (unique IDs for example won't be very useful to visualize, for example), for some categorical features, let's see some bar plot visualizations. For simplicity and speed, here we only show box plots for those features with less than 50 unique values.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for c in categorical_features_all:
    if len(df[c].value_counts()) < 50:
        print(c)
        df[c].value_counts().plot.bar()
        plt.show()

__Histograms:__ Histograms show distribution of numeric data. Data is divided into "buckets" or "bins".

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for c in numerical_features_all:
    print(c)
    df[c].plot.hist(bins=5)
    plt.show()

If for some histograms the values are heavily placed in the first bin, it is good to check for outliers, either checking the min-max values of those particular features and/or explore value ranges.

In [ ]:
for c in numerical_features_all:
    print(c)
    print('min:', df[c].min(), 'max:', df[c].max())

With __value_counts()__ function, we can increase the number of histogram bins to 10 for more bins for a more refined view of the numerical features.

In [ ]:
for c in numerical_features_all: 
    print(c)
    print(df[c].value_counts(bins=10, sort=False))
    plt.show()

If any outliers are identified as very likely wrong values, dropping them could improve the numerical values histograms, and later overall model performance. While a good rule of thumb is that anything not in the range of (Q1 - 1.5 IQR) and (Q3 + 1.5 IQR) is an outlier, other rules for removing 'outliers' should be considered as well. For example, removing any values in the upper 1%. 

In [ ]:
for c in numerical_features_all:
    print(c)
    
    # Drop values below Q1 - 1.5 IQR and beyond Q3 + 1.5 IQR
    #Q1 = df[c].quantile(0.25)
    #Q3 = df[c].quantile(0.75)
    #IQR = Q3 - Q1
    #print (Q1 - 1.5*IQR, Q3 + 1.5*IQR)
    
    #dropIndexes = df[df[c] > Q3 + 1.5*IQR].index
    #df.drop(dropIndexes , inplace=True)
    #dropIndexes = df[df[c] < Q1 - 1.5*IQR].index
    #df.drop(dropIndexes , inplace=True)
    
    # Drop values beyond 90% of max()
    dropIndexes = df[df[c] > df[c].max()*9/10].index
    df.drop(dropIndexes , inplace=True)


In [ ]:
for c in numerical_features_all:
    print(c)
    print(df[c].value_counts(bins=10, sort=False))
    plt.show()

Let's see the histograms again, with more bins for vizibility.

In [ ]:
for c in numerical_features_all:
    print(c)
    df[c].plot.hist(bins=100)
    plt.show()

## 4. <a name="4">Scatter Plots and Correlation</a>
(<a href="#0">Go to top</a>)

### Scatter plot
Scatter plots are simple 2D plots of two numerical variables that can be used to examine the relationship between two variables. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(numerical_features_all), len(numerical_features_all), figsize=(16, 16), sharex=False, sharey=False)
for i in range(0,len(numerical_features_all)):
    for j in range(0,len(numerical_features_all)):
        axes[i,j].scatter(x = df[numerical_features_all[i]], y = df[numerical_features_all[j]])
fig.tight_layout()

### Scatterplot with Identification

We can also add the target values, 0 or 1, to our scatter plot.

In [ ]:
import seaborn as sns

X1 = df[[numerical_features_all[0], numerical_features_all[1]]][df[model_target] == 0]
X2 = df[[numerical_features_all[0], numerical_features_all[1]]][df[model_target] == 1]

plt.scatter(X1.iloc[:,0], 
            X1.iloc[:,1], 
            s=50, 
            c='blue', 
            marker='o', 
            label='0')

plt.scatter(X2.iloc[:,0], 
            X2.iloc[:,1], 
            s=50, 
            c='red', 
            marker='v', 
            label='1')

plt.xlabel(numerical_features_all[0])
plt.ylabel(numerical_features_all[1])
plt.legend()
plt.grid()
plt.show()

Scatterplots with identification, can sometimes help identify whether or not we can get good separation between the data points, based on these two numerical features alone. 

### Correlation Matrix Heatmat
We plot the correlation matrix. Correlation scores are calculated for numerical fields. 

In [ ]:
cols=[numerical_features_all[0], numerical_features_all[1]]
#print(df[cols].corr())
df[cols].corr().style.background_gradient(cmap='tab20c')

Similar to scatterplots, but now the correlation matrix values can more clearly pinpoint relationships between the numerical features. Correlation values of -1 means perfect negative correlation, 1 means perfect positive correlation, and 0 means there is no relationship between the two numerical features.

### A fancy example using Seaborn

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

# Generate a large random dataset
rs = np.random.RandomState(33)
d = pd.DataFrame(data=rs.normal(size=(100, 26)),
                 columns=list(ascii_letters[26:]))

# Compute the correlation matrix
corr = d.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Also, more exploratory data analysis might reveal other important hidden atributes and/or relationships of the model features considered. 

## 5. <a name="5">Handling Missing Values</a>
(<a href="#0">Go to top</a>)

  * <a href="#51">Drop columns with missing values</a>
  * <a href="#52">Drop rows with missing values</a>
  * <a href="#53"> Impute (fill-in) missing values with .fillna()</a>
  * <a href="#54"> Impute (fill-in) missing values with sklearn's SimpleImputer</a>

Let's first check the number of missing (nan) values for each column.

In [ ]:
df.isna().sum()

Let's explore a few options dealing with missing values, when there are values missing on many features, both numerical and categorical types. 

### <a name="51">Drop columns with missing values</a>
(<a href="#5">Go to Handling Missing Values</a>)

We can drop some feautures/columns if we think there is significant amount of missing data in those features. Here we 
are dropping features having more than 20% missing values.

__Hint:__ You can also use __inplace=True__ parameter to drop features inplace without assignment.


In [ ]:
threshold = 2/10
print((df.isna().sum()/len(df.index)))
columns_to_drop = df.loc[:,list(((df.isna().sum()/len(df.index))>=threshold))].columns    
print(columns_to_drop)

df_columns_dropped = df.drop(columns_to_drop, axis = 1)  
df_columns_dropped.head()

In [ ]:
df_columns_dropped.isna().sum()

In [ ]:
df_columns_dropped.shape

Note the reduced size of the dataset features. This can sometimes lead to underfitting models -- not having enough features to build a good model able to capture the pattern in the dataset, especially when dropping features that are essential to the task at hand. 

### <a name="52">Drop rows with missing values</a>
(<a href="#5">Go to Handling Missing Values</a>)

Here, we simply drop rows that have at least one missing value. There are other drop options to explore, depending on specific problems.

In [ ]:
df_missing_dropped = df.dropna()

Let's check the missing values below.

In [ ]:
df_missing_dropped.isna().sum()

In [ ]:
df_missing_dropped.shape

In [ ]:
# why did lose so many records?  What should we have done differently?


This approach can dramatically reduce the number of data samples. This can sometimes lead to overfitting models -- especially when the number of features is greater or comparable to the number of data samples. 

### <a name="53">Impute (fill-in) missing values with .fillna()</a>
(<a href="#5">Go to Handling Missing Values</a>)

Rather than dropping rows (data samples) and/or columns (features), another strategy to deal with missing values would be to actually complete the missing values with new values: imputation of missing values.

__Imputing Numerical Values:__ The easiest way to impute numerical values is to get the __average (mean) value__ for the corresponding column and use that as the new value for each missing record in that column. 

In [ ]:
# Impute numerical features by using the mean per feature to replace the nans

# Assign our df to a new df 
df_imputed = df.copy()
print(df_imputed[numerical_features_all].isna().sum())

# Impute our two numerical features with the means. 
df_imputed[numerical_features_all] = df_imputed[numerical_features_all].fillna(df_imputed[numerical_features_all].mean())

print(df_imputed[numerical_features_all].isna().sum())

__Imputing Categorical Values:__ We can impute categorical values by getting the most common (mode) value for the corresponding column and use that as the new value for each missing record in that column. 

In [ ]:
# Impute categorical features by using the mode per feature to replace the nans

# Assign our df to a new df 
df_imputed_c = df.copy()
print(df_imputed_c[categorical_features_all].isna().sum())

for c in categorical_features_all:
    # Find the mode per each feature
    mode_impute = df_imputed_c[c].mode()
    print(c, mode_impute)

    # Impute our categorical features with the mode
    # "inplace=True" parameter replaces missing values in place (no need for left handside assignment)
    
    # df_imputed_c[c].fillna(False, inplace=True)
    df_imputed_c[c].fillna(mode_impute, inplace=True)
    # instead of False, I think this should be the value your want to imput with, in this case "mode_impute"
    
print(df_imputed_c[categorical_features_all].isna().sum())

We can also create a new category, such as "Missing", for alll or elected categorical features.

In [ ]:
# Impute categorical features by using a placeholder value

# Assign our df to a new df 
df_imputed = df.copy()
print(df_imputed[categorical_features_all].isna().sum())

# Impute our categorical features with a new category named "Missing". 
df_imputed[categorical_features_all]= df_imputed[categorical_features_all].fillna("Missing")

print(df_imputed[categorical_features_all].isna().sum())

### <a name="54">Impute (fill-in) missing values with sklearn's __SimpleImputer__</a>
(<a href="#5">Go to Handling Missing Values</a>)

A more elegant way to implement imputation is using sklearn's __SimpleImputer__, a class implementing .fit() and .transform() methods.


In [ ]:
# Impute numerical columns by using the mean per column to replace the nans

from sklearn.impute import SimpleImputer

# Assign our df to a new df
df_sklearn_imputed = df.copy()
print(df_sklearn_imputed[numerical_features_all].isna().sum())

imputer = SimpleImputer(strategy='mean')
df_sklearn_imputed[numerical_features_all] = imputer.fit_transform(df_sklearn_imputed[numerical_features_all])

print(df_sklearn_imputed[numerical_features_all].isna().sum())

In [ ]:
# Impute categorical columns by using the mode per column to replace the nans

# Pick some categorical features you desire to impute with this approach
categoricals_missing_values = df[categorical_features_all].loc[:,list(((df[categorical_features_all].isna().sum()/len(df.index)) > 0.0))].columns    
columns_to_impute = categoricals_missing_values[1:3]
print(columns_to_impute)

from sklearn.impute import SimpleImputer

# Assign our df to a new df
df_sklearn_imputer = df.copy()
print(df_sklearn_imputer[columns_to_impute].isna().sum())

imputer = SimpleImputer(strategy='most_frequent')
df_sklearn_imputer[columns_to_impute] = imputer.fit_transform(df_sklearn_imputer[columns_to_impute])

print(df_sklearn_imputer[columns_to_impute].isna().sum())

In [ ]:
# Impute categorical columns by using a placeholder "Missing"

# Pick some categorical features you desire to impute with this approach
categoricals_missing_values = df[categorical_features_all].loc[:,list(((df[categorical_features_all].isna().sum()/len(df.index)) > 0.0))].columns    
columns_to_impute = categoricals_missing_values[1:3]
print(columns_to_impute)

from sklearn.impute import SimpleImputer

# Assign our df to a new df
df_sklearn_imputer = df.copy()
print(df_sklearn_imputer[columns_to_impute].isna().sum())

imputer = SimpleImputer(strategy='constant', fill_value = "Missing")
df_sklearn_imputer[columns_to_impute] = imputer.fit_transform(df_sklearn_imputer[columns_to_impute])

print(df_sklearn_imputer[columns_to_impute].isna().sum())